# FEATURE ENGINEERING

#### Load data + basic cleaning from EDA

In [90]:
import pandas as pd

df = pd.read_csv('../data/interim/data_cleaned.csv')
df_test = pd.read_csv('../data/raw/test.csv')

# Drop unused columns (after EDA and Feature importance)
drop_cols = [
    'id',
    'age',
    'gender',
    'internet_access'
    # 'course',
    # 'exam_difficulty',
]

df = df.drop(columns=drop_cols, errors='ignore')
df_test = df_test.drop(columns=drop_cols, errors='ignore')

target = 'exam_score'

#### Define target and feature groups

In [91]:
num_base = [
    'study_hours',
    'class_attendance',
    'sleep_hours'
]

cat_ordinal = [
    'sleep_quality',
    'facility_rating',
    'exam_difficulty'
]

cat_onehot = [
    'study_method',
    'course'
]

#### Feature interaction

In [92]:
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df['high_study'] = (df['study_hours'] > 6).astype(int)
    df['low_attendance'] = (df['class_attendance'] < 60).astype(int)

    df['study_efficiency'] = df['study_hours'] * df['class_attendance']
    df['sleep_attend_product'] = df['sleep_hours'] * df['class_attendance']

    df['sleep_deviation'] = (df['sleep_hours'] - 7).abs()

    return df

## Feature Encoding (categorical) and Scaling (numerical)

In [93]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

sleep_quality_order = ['poor', 'average', 'good']
facility_rating_order = ['low', 'medium', 'high']
exam_difficulty_order = ['easy', 'moderate', 'hard']

num_features = num_base + [
    'high_study',
    'low_attendance',
    'study_efficiency',
    'sleep_attend_product',
    'sleep_deviation'
]

preprocessor = ColumnTransformer(
    transformers=[
        # Numerical (scaled)
        ('num', StandardScaler(), num_features),

        # Ordinal categorical
        ('ord', OrdinalEncoder(
            categories=[
                sleep_quality_order,
                facility_rating_order,
                exam_difficulty_order
            ]
        ), cat_ordinal),

        # One-hot categorical
        ('onehot', OneHotEncoder(
            drop='first',
            handle_unknown='ignore',
            sparse_output=False
        ), cat_onehot)
    ],
    remainder='drop'
)

## Pipeline

In [94]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

feature_engineering = FunctionTransformer(add_features, validate=False)

pipeline = Pipeline(steps=[
    ('feature_engineering', feature_engineering),
    ('preprocessor', preprocessor)
])

#### Fit on train

In [95]:
X_train = df.drop(columns=[target])
y_train = df[target]

pipeline.fit(X_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('feature_engineering', ...), ('preprocessor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"func func: callable, default=NoneThe callable to use for the transformation. This will be passedthe same arguments as transform, with args and kwargs forwarded.If func is None, then func will be the identity function.",<function add...0025465E9A200>
,"inverse_func inverse_func: callable, default=NoneThe callable to use for the inverse transformation. This will bepassed the same arguments as inverse transform, with args andkwargs forwarded. If inverse_func is None, then inverse_funcwill be the identity function.",None
,"validate validate: bool, default=FalseIndicate that the input X array should be checked before calling``func``. The possibilities are:- If False, there is no input validation.- If True, then X will be converted to a 2-dimensional NumPy array or sparse matrix. If the conversion is not possible an exception is raised... versionchanged:: 0.22 The default of ``validate`` changed from True to False.",False
,"accept_sparse accept_sparse: bool, default=FalseIndicate that func accepts a sparse matrix as input. If validate isFalse, this has no effect. Otherwise, if accept_sparse is false,sparse matrix inputs will cause an exception to be raised.",False
,"check_inverse check_inverse: bool, default=TrueWhether to check that or ``func`` followed by ``inverse_func`` leads tothe original inputs. It can be used for a sanity check, raising awarning when the condition is not fulfilled... versionadded:: 0.20",True
,"feature_names_out feature_names_out: callable, 'one-to-one' or None, default=NoneDetermines the list of feature names that will be returned by the`get_feature_names_out` method. If it is 'one-to-one', then the outputfeature names will be equal to the input feature names. If it is acallable, then it must take two positional arguments: this`FunctionTransformer` (`self`) and an array-like of input feature names(`input_features`). It must return an array-like of output featurenames. The `get_feature_names_out` method is only defined if`feature_names_out` is not None.See ``get_feature_names_out`` for more details... versionadded:: 1.1",None
,"kw_args kw_args: dict, default=NoneDictionary of additional keyword argu

In [96]:
import pandas as pd

feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()

feature_names = pd.Series(feature_names)

feature_names = (
    feature_names
    .str.replace('num__', '', regex=False)
    .str.replace('ord__', '', regex=False)
    .str.replace('onehot__', '', regex=False)
    .str.replace(' ', '_')
)

In [97]:
X_train = df.drop(columns=[target])
y_train = df[target]

pipeline.fit(X_train)

X_train = pipeline.transform(X_train)
X_test = pipeline.transform(df_test)

feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
feature_names = (
    pd.Series(feature_names)
    .str.replace('num__', '', regex=False)
    .str.replace('ord__', '', regex=False)
    .str.replace('onehot__', '', regex=False)
)

train_df = pd.DataFrame(X_train, columns=feature_names)
train_df[target] = y_train.values

test_df = pd.DataFrame(X_test, columns=feature_names)

train_df.to_csv('../data/processed/dataset_train.csv', index=False)
test_df.to_csv('../data/processed/dataset_test.csv', index=False)

import joblib
joblib.dump(pipeline, '../models/fe_pipeline.joblib')

['../models/fe_pipeline.joblib']